In [162]:
import transformers
import torch
import numpy as np
import pandas as pd
import random
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from datasets import load_dataset, load_metric
from transformers import Trainer,TrainingArguments,AutoTokenizer,AutoModelForSequenceClassification,DataCollatorForTokenClassification,EarlyStoppingCallback, BertTokenizer
from datasets import Dataset
import time

In [163]:
model_checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
batch_size = 32
wd = 0.01
lr = 2e-5
epochs = 5

task = "binary_classification"
label_list = [0,1]
num_labels = 2

In [164]:
def seed_everything(seed:42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
    
# fix seed
seed_everything(42)

In [165]:
df = pd.read_csv('YOUR PAYLOAD data !!!!!')
df.head()

,payload,label,ips_00001_payload_base64,ips_00001_payload_sql_comb_01,ips_00001_payload_sql_comb_02,ips_00001_payload_sql_comb_03,ips_00001_payload_xss_comb_01,ips_00001_payload_cmd_comb_01,ips_00001_payload_log4j_comb_01,ips_00001_payload_word_comb_01,ips_00001_payload_word_comb_02,ips_00001_payload_word_comb_03,ips_00001_payload_wp_comb_01,ips_00001_payload_word_comb_04,ips_00001_payload_useragent_comb,ips_00001_payload_whitelist
0,NaN,normal,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,NaN,normal,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,NaN,normal,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,NaN,normal,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,NaN,normal,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [166]:
df = df.sample(10000)

In [167]:
df.isna().sum()

payload                             365
label                                 0
ips_00001_payload_base64              0
ips_00001_payload_sql_comb_01         0
ips_00001_payload_sql_comb_02         0
ips_00001_payload_sql_comb_03         0
ips_00001_payload_xss_comb_01         0
ips_00001_payload_cmd_comb_01         0
ips_00001_payload_log4j_comb_01       0
ips_00001_payload_word_comb_01        0
ips_00001_payload_word_comb_02        0
ips_00001_payload_word_comb_03        0
ips_00001_payload_wp_comb_01          0
ips_00001_payload_word_comb_04        0
ips_00001_payload_useragent_comb      0
ips_00001_payload_whitelist           0
dtype: int64

In [168]:
df['payload'] = df['payload'].fillna('')

In [169]:
df['label'] = np.where(df['label'] == 'anomalies', 1, 0)
df['label'].value_counts()

1    8403
0    1597
Name: label, dtype: int64

In [170]:
df = df[['payload', 'label']]

In [171]:
df['label'].value_counts()

1    8403
0    1597
Name: label, dtype: int64

In [172]:
train_data, val_data = train_test_split(df, random_state=42,test_size=.2)

In [173]:
print(train_data.shape)
print(val_data.shape)

(8000, 2)
(2000, 2)


In [174]:
df_train = pd.DataFrame({"payload":train_data["payload"],'label':train_data["label"]})
dataset_train = Dataset.from_pandas(df_train)

In [175]:
df_val = pd.DataFrame({"payload": val_data["payload"],'label':val_data["label"]})
dataset_val = Dataset.from_pandas(df_val)

In [176]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

loading configuration file https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english/resolve/main/config.json from cache at /Users/choiwb/.cache/huggingface/transformers/4e60bb8efad3d4b7dc9969bf204947c185166a0a3cf37ddb6f481a876a3777b5.9f8326d0b7697c7fd57366cdde57032f46bc10e37ae81cb7eb564d66d23ec96b
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased-finetuned-sst-2-english",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "sst-2",
  "hidden_dim": 3072,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie

In [177]:
model.resize_token_embeddings(len(tokenizer))

Embedding(30522, 768, padding_idx=0)

In [178]:
ai_field = ['select##from', 'select##count', 'select##distinct', 'union##select', 'select##extractvalue##xmltype',
           'from##generate##series', 'from##group##by', 'case##when', 'then##else', 'waitfor##delay', 'db##sql##server',
           'cast##chr', 'like##http/1.', 'upper##xmltype', 'script##alert', 'wget##ttp', 'chmod##777', 'rm##rf', 'cd##tmp',
           'jndi##dap', 'jndi##dns', 'etc##passwd', 'document##createelement', 'cgi##bin', 'document##forms', 'document##location',
           'fckeditor##filemanager', 'manager##html', 'current_config##passwd', 'currentsetting##htm', 'well##known',
           'bash##history', 'apache##struts', 'document##open', 'backup##sql', 'robots##txt', 'sqlexec##php',
           'exec', 'htaccess', 'htpasswd', 'cgi##cgi', 'api##ping', 'aaaaaaaaaa', 'cacacacaca', 'mozi.',
           'bingbot', 'md5', 'jpg##http##1.1', 'count##cgi##http', 'this##program##can', 'sleep##sleep', 'and##sleep',
           'delete', 'get##ping', 'msadc##dll##http', 'filename##asp', 'filename##jsp',
           'wp##login', 'wp##content', 'wp##include', 'wp##config', 'cmd##open', 'echo##shellshock', 'php##echo',
           'echo', 'admin##php', 'script##setup##php', 'phpinfo', 'adminostrator', 'phpmyadmin', 'access', 'passwd', 'eval', 'php', 'cmd', 'mdb',
           'wise##survey##admin', 'admin##serv##admpw', 'php##create##function',
           'user-agent##zgrab', 'user-agent##nmap', 'user-agent##dirbuster', 'user-agent##ahrefsbot',
           'user-agent##baiduspider', 'user-agent##mj12bot', 'user-agent##petalbot',
           'user-agent##semrushbot', 'user-agent##curl', 'user-agent##masscan', 'user-agent##sqlmap',
           'user-agent##urlgrabber##yum']

In [179]:
# ai_list에 element 안에 '(.*?)'가 포함되어 있는 경우, '(.*?)' 기준으로 split 후, 리스트에 추가
first_ai_list = [x.split('##')[0] for x in ai_field if '##' in x]
end_ai_list = [x.split('##')[1] for x in ai_field if '##' in x]
except_ai_list = [x.replace('[\\.]', '.') for x in ai_field]
# ai_list의 element 안에 ('*?)' 가 2번 포함되어 있는 경우, 2번째 '(.*?)' 기준으로 split 후, 리스트에 추가
two_ai_list = [x.split('##')[2] for x in ai_field if x.count('##') == 2]
ai_list_split = first_ai_list + end_ai_list + ai_field + except_ai_list

# ai_list_split 안에 중복되는 element 가 있는 경우, 단일 처리
ai_list_split = list(set(ai_list_split))

# ai_list_split 안에 '(.*?' 나, '[\\.]' 가 포함되어 있는 경우, 제거
ai_list_split = [x for x in ai_list_split if '##' not in x]
ai_list_split = [x for x in ai_list_split if '[\\.]' not in x]

In [180]:
print(ai_list_split)
print(len(ai_list_split))

['jsp', 'wget', 'from', 'admin', 'delete', 'cd', 'bin', 'select', 'serv', 'cacacacaca', 'manager', 'msadc', 'well', 'asp', 'curl', 'htpasswd', 'jndi', 'dap', 'aaaaaaaaaa', 'group', 'bash', 'sqlmap', 'dns', 'sleep', 'robots', 'history', 'ping', 'apache', 'like', 'bingbot', 'http/1.', 'current_config', 'masscan', 'tmp', 'mdb', 'chr', 'phpmyadmin', 'sqlexec', 'delay', 'mozi.', 'dll', 'rf', 'generate', 'then', 'cast', 'create', 'dirbuster', 'jpg', 'backup', 'passwd', 'struts', 'open', 'adminostrator', 'case', 'filemanager', 'program', 'semrushbot', 'location', 'txt', 'ahrefsbot', 'config', 'urlgrabber', 'chmod', 'forms', 'access', 'api', 'fckeditor', 'wise', 'ttp', 'waitfor', 'eval', 'exec', 'http', 'distinct', 'htm', 'setup', 'login', 'rm', 'md5', 'upper', 'survey', 'content', 'include', 'nmap', 'union', 'when', 'etc', 'htaccess', '777', 'extractvalue', 'script', 'createelement', 'cmd', 'html', 'filename', 'and', 'currentsetting', 'alert', 'xmltype', 'user-agent', 'phpinfo', 'cgi', 'zgrab

In [181]:
# ai_list_split = set(ai_list_split)
len(ai_list_split)

118

In [182]:
new_tokens = set(ai_list_split) - set(tokenizer.vocab.keys())
len(new_tokens)

63

In [183]:
new_tokens

{'777',
 'aaaaaaaaaa',
 'admin',
 'adminostrator',
 'ahrefsbot',
 'asp',
 'baiduspider',
 'bingbot',
 'cacacacaca',
 'cgi',
 'chmod',
 'chr',
 'cmd',
 'config',
 'createelement',
 'current_config',
 'currentsetting',
 'dap',
 'delete',
 'dirbuster',
 'dll',
 'dns',
 'eval',
 'exec',
 'extractvalue',
 'fckeditor',
 'filemanager',
 'filename',
 'htaccess',
 'htm',
 'htpasswd',
 'http/1.',
 'jndi',
 'jpg',
 'jsp',
 'login',
 'masscan',
 'md5',
 'mdb',
 'mj12bot',
 'mozi.',
 'msadc',
 'nmap',
 'passwd',
 'petalbot',
 'phpinfo',
 'phpmyadmin',
 'semrushbot',
 'serv',
 'shellshock',
 'sqlexec',
 'sqlmap',
 'struts',
 'tmp',
 'ttp',
 'txt',
 'urlgrabber',
 'user-agent',
 'waitfor',
 'wget',
 'wp',
 'xmltype',
 'zgrab'}

In [184]:
# add the tokens to the tokenizer vocabulary
# tokenizer.add_tokens(list(new_tokens))
tokenizer.add_special_tokens({"additional_special_tokens": ai_list_split})

Assigning ['jsp', 'wget', 'from', 'admin', 'delete', 'cd', 'bin', 'select', 'serv', 'cacacacaca', 'manager', 'msadc', 'well', 'asp', 'curl', 'htpasswd', 'jndi', 'dap', 'aaaaaaaaaa', 'group', 'bash', 'sqlmap', 'dns', 'sleep', 'robots', 'history', 'ping', 'apache', 'like', 'bingbot', 'http/1.', 'current_config', 'masscan', 'tmp', 'mdb', 'chr', 'phpmyadmin', 'sqlexec', 'delay', 'mozi.', 'dll', 'rf', 'generate', 'then', 'cast', 'create', 'dirbuster', 'jpg', 'backup', 'passwd', 'struts', 'open', 'adminostrator', 'case', 'filemanager', 'program', 'semrushbot', 'location', 'txt', 'ahrefsbot', 'config', 'urlgrabber', 'chmod', 'forms', 'access', 'api', 'fckeditor', 'wise', 'ttp', 'waitfor', 'eval', 'exec', 'http', 'distinct', 'htm', 'setup', 'login', 'rm', 'md5', 'upper', 'survey', 'content', 'include', 'nmap', 'union', 'when', 'etc', 'htaccess', '777', 'extractvalue', 'script', 'createelement', 'cmd', 'html', 'filename', 'and', 'currentsetting', 'alert', 'xmltype', 'user-agent', 'phpinfo', 'cg

63

In [185]:
# add new, random embeddings for the new tokens
model.resize_token_embeddings(len(tokenizer))

Embedding(30585, 768)

In [186]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [187]:
def tokenize_function(examples):
    return tokenizer(examples["payload"], padding="max_length", max_length = 250, truncation=True)

In [188]:
tokenized_train_datasets = dataset_train.map(tokenize_function, batched=True)
tokenized_val_datasets = dataset_val.map(tokenize_function, batched=True)

100%|██████████| 2/2 [00:00<00:00, 10.97ba/s]


In [189]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate= lr,#2e-5
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    seed = 42,
    weight_decay=wd,load_best_model_at_end=True,)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [190]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [191]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_val_datasets,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],   
)

In [34]:
# gpu 또는 mps 자동 학습 시작.
# trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: payload, __index_level_0__. If payload, __index_level_0__ are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/Users/choiwb/miniforge3/envs/newenv/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8000
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1250
 20%|██        | 250/1250 [20:53<1:18:55,  4.74s/it]The following columns in the evaluation set don't h

{'eval_loss': 0.0987917110323906, 'eval_accuracy': 0.9755, 'eval_f1': 0.9552710551238603, 'eval_precision': 0.9687415436423223, 'eval_recall': 0.9429893496294481, 'eval_runtime': 70.9422, 'eval_samples_per_second': 28.192, 'eval_steps_per_second': 0.888, 'epoch': 1.0}


Model weights saved in test-binary_classification/checkpoint-250/pytorch_model.bin
tokenizer config file saved in test-binary_classification/checkpoint-250/tokenizer_config.json
Special tokens file saved in test-binary_classification/checkpoint-250/special_tokens_map.json
 40%|████      | 500/1250 [40:27<54:12,  4.34s/it]  The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: payload, __index_level_0__. If payload, __index_level_0__ are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 32


{'loss': 0.1128, 'learning_rate': 1.2e-05, 'epoch': 2.0}


                                                  
 40%|████      | 500/1250 [41:36<54:12,  4.34s/it]Saving model checkpoint to test-binary_classification/checkpoint-500
Configuration saved in test-binary_classification/checkpoint-500/config.json


...

Model weights saved in test-binary_classification/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-binary_classification/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-binary_classification/checkpoint-1000/special_tokens_map.json
100%|██████████| 1250/1250 [1:33:25<00:00,  3.95s/it]  The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: payload, __index_level_0__. If payload, __index_level_0__ are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 32
                                                     
100%|██████████| 1250/1250 [1:34:33<00:00,  3.95s/it]Saving model checkpoint to test-binary_classification/checkpoint-1250
Configuration saved in test-binary_classification/checkpoint-1250/config.json


{'eval_loss': 0.10543133318424225, 'eval_accuracy': 0.9785, 'eval_f1': 0.9614079631526823, 'eval_precision': 0.9664579931046997, 'eval_recall': 0.9565345182469955, 'eval_runtime': 68.1362, 'eval_samples_per_second': 29.353, 'eval_steps_per_second': 0.925, 'epoch': 5.0}


Model weights saved in test-binary_classification/checkpoint-1250/pytorch_model.bin
tokenizer config file saved in test-binary_classification/checkpoint-1250/tokenizer_config.json
Special tokens file saved in test-binary_classification/checkpoint-1250/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from test-binary_classification/checkpoint-500 (score: 0.07354895770549774).
100%|██████████| 1250/1250 [1:34:35<00:00,  4.54s/it]

{'train_runtime': 5675.083, 'train_samples_per_second': 7.048, 'train_steps_per_second': 0.22, 'train_loss': 0.061591992568969724, 'epoch': 5.0}


TrainOutput(global_step=1250, training_loss=0.061591992568969724, metrics={'train_runtime': 5675.083, 'train_samples_per_second': 7.048, 'train_steps_per_second': 0.22, 'train_loss': 0.061591992568969724, 'epoch': 5.0})

In [35]:
# trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: payload, __index_level_0__. If payload, __index_level_0__ are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 32
100%|██████████| 63/63 [01:07<00:00,  1.07s/it]


{'eval_loss': 0.07354895770549774,
 'eval_accuracy': 0.98,
 'eval_f1': 0.9641432477253373,
 'eval_precision': 0.968640881617699,
 'eval_recall': 0.9597853942342016,
 'eval_runtime': 68.473,
 'eval_samples_per_second': 29.209,
 'eval_steps_per_second': 0.92,
 'epoch': 5.0}

In [36]:
# 학습 모델 저장
import os
model_save_path = 'BERT model save path !!!!!'
model_dir = os.path.join(model_save_path, 'BERT_transfer_model.pkl')
# torch.save(model, model_dir)
# orch.save(model.state_dict(), model_dir)

In [37]:
# pytorch 모델 호출
load_model = torch.load(model_dir)
# model.load_state_dict(torch.load(model_dir))
# model.eval()
load_model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30585, 768)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Li

In [38]:
device = torch.device('mps')
device

device(type='mps')

In [39]:
load_model = load_model.to(device)
load_model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30585, 768)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Li

In [250]:
sample_df = df[df['payload'].str.contains('etc', case = False)]
sample_df = sample_df[sample_df['payload'].str.contains('passwd', case = False)]

sample_df = sample_df.sample(1)
sample_df.iloc[0,0]

'GET /bbs/board.php?bo_table=ad_04_01%27)%20AND%201=1%20UNION%20ALL%20SELECT%201,NULL,%27%3Cscript%3Ealert(₩₩₩%22XSS₩₩₩%22)%3C/script%3E%27,table_name%20FROM%20information_schema.tables%20WHERE%202%3E1--/**/;%20EXEC%20xp_cmdshell(%27cat%20../../../etc/passwd%27) HTTP/1.1\naccept: */*\nuser-agent: Mozilla/5.0 (Windows NT 8.0; Win64; x64; rv:39.51.2) Gecko/20100101 Firefox/39.51.2\nhost: webpage.com'

In [251]:
import scipy as sp
import shap

# define a prediction function
def bert_predict(x):
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=250, truncation=True) for v in x]).to(device)

    # outputs = model(tv)[0].detach().cpu().numpy()
    outputs = load_model(tv)[0].detach().cpu().numpy()

    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores[:,1]) # use one vs rest logit units

    return val


In [252]:
# "/" 또는 "HTTP/1.1" 또는 "\s" 또는 "?" 기준으로 분할 regex
masker = shap.maskers.Text(tokenizer = r"(\s|%20|/|%2F|HTTP/1.1|\?)")

In [257]:
pytorch_bert_explainer = shap.Explainer(model = bert_predict, masker = tokenizer)
pytorch_bert_explainer_custom = shap.Explainer(model = bert_predict, masker = masker)

In [258]:
bert_shap_values = pytorch_bert_explainer(sample_df['payload'], fixed_context=1, batch_size=1)
bert_shap_values_custom = pytorch_bert_explainer_custom(sample_df['payload'], fixed_context=1, batch_size=1)

In [259]:
bert_shap_values.data

(array(['', 'GET ', '/', 'bb', 's', '/', 'board', '.', 'php', '?', 'bo',
        '_', 'table', '=', 'ad', '_', '04', '_', '01', '%', '27', ')', '%',
        '20', 'AND', '%', '201', '=', '1', '%', '20', 'UNI', 'ON', '%',
        '20', 'ALL', '%', '20S', 'ELE', 'CT', '%', '201', ',', 'NULL', ',',
        '%', '27', '%', '3', 'C', 'script', '%', '3', 'E', 'alert', '(',
        '₩', '₩', '₩', '%', '22', 'X', 'SS', '₩', '₩', '₩', '%', '22', ')',
        '%', '3', 'C', '/', 'script', '%', '3', 'E', '%', '27', ',',
        'table', '_', 'name', '%', '20', 'FR', 'OM', '%', '20', 'in', 'fo',
        'rm', 'at', 'ion', '_', 'sc', 'hema', '.', 'tables', '%', '20',
        'W', 'HER', 'E', '%', '202', '%', '3', 'E', '1', '-', '-', '/',
        '*', '*', '/', ';', '%', '20', 'EX', 'EC', '%', '20', 'x', 'p',
        '_', 'cmd', 'shell', '(', '%', '27', 'cat', '%', '20', '.', '.',
        '/', '.', '.', '/', '.', '.', '/', 'etc', '/', 'passwd', '%', '27',
        ') ', 'HTTP', '/', '1', '.', '1\n', 

In [260]:
bert_shap_values_custom.data

(array(['GET ', '/', 'bbs/', 'board.php?', 'bo_table=ad_04_01%27)%20',
        'AND%20', '1=1%20', 'UNION%20', 'ALL%20', 'SELECT%20',
        '1,NULL,%27%3Cscript%3Ealert(₩₩₩%22XSS₩₩₩%22)%3C/',
        'script%3E%27,table_name%20', 'FROM%20',
        'information_schema.tables%20', 'WHERE%20', '2%3E1--/', '**/',
        ';%20', 'EXEC%20', 'xp_cmdshell(%27cat%20', '../', '../', '../',
        'etc/', 'passwd%27) ', 'HTTP/1.1', '\n', 'accept: ', '*/', '*\n',
        'user-agent: ', 'Mozilla/', '5.0 ', '(Windows ', 'NT ', '8.0; ',
        'Win64; ', 'x64; ', 'rv:39.51.2) ', 'Gecko/', '20100101 ',
        'Firefox/', '39.51.2\n', 'host: ', 'webpage.com'], dtype=object),)

In [261]:
shap.text_plot(bert_shap_values, display = True)

In [262]:
shap.text_plot(bert_shap_values_custom, display = True)